# Using Embedding with Cosine Similarity

In [1]:
# Import packages

import numpy as np
import pandas as pd
import csv
import numpy as np
import scipy
import six
import smart_open as so
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import string
import re

In [3]:
# Read in the data
df2 = pd.read_pickle('concat_df.pkl')

## Prep the data

In [88]:
df2.head(3)

,id,name,blurb,goal,pledged,state,country,currency_symbol,deadline,state_changed_at,...,backers_count,location.id,location.name,category.id,category.name,category.slug,spotlight,urls.web.project,run_id,cleanblurb
0,287514992,New Final Round Album,2006 was almost 7 years ago.... Can you believ...,200.0,802.0,successful,US,$,1391899046,1391899046,...,21,2379574.0,Chicago,43,Rock,music/rock,True,https://www.kickstarter.com/projects/149592564...,Kickstarter_2019-03-14T03_20_12_200Z,"[2006, was, almost, 7, years, ago, ..., ., can..."
1,385129759,Princess Pals Enamel Pin Series,An adorable fantasy enamel pin series of princ...,400.0,2259.0,successful,US,$,1551801611,1551801611,...,97,2486340.0,Sacramento,54,Mixed Media,art/mixed media,True,https://www.kickstarter.com/projects/freckledd...,Kickstarter_2019-03-14T03_20_12_200Z,"[an, adorable, fantasy, enamel, pin, series, o..."
2,681033598,Their Life Through Their Lens-the Amish and Me...,Helping a community come together to set the s...,27224.0,29638.0,successful,US,$,1480607930,1480607932,...,88,2383660.0,Columbus,280,Photobooks,photography/photobooks,True,https://www.kickstarter.com/projects/119685626...,Kickstarter_2019-03-14T03_20_12_200Z,"[helping, a, community, come, together, to, se..."


In [89]:
df2.shape

(1668216, 23)

In [95]:
# Drop duplicates
df_nd = df2.sort_values('run_id').drop_duplicates(subset = 'id', keep = 'last')

In [96]:
df_nd.shape

(170001, 23)

In [97]:
df_nd.head(3)

,id,name,blurb,goal,pledged,state,country,currency_symbol,deadline,state_changed_at,...,backers_count,location.id,location.name,category.id,category.name,category.slug,spotlight,urls.web.project,run_id,cleanblurb
1581794,2061552595,Scott Fagan Sings Stephin Merritt,A tribute to songwriter Stephin Merritt (Magne...,8000.0,3565.0,failed,US,$,1391977914,1391977918,...,83,2459115.0,New York,42,Pop,music/pop,False,https://www.kickstarter.com/projects/450132630...,Kickstarter_2018-08-16T03_20_13_856Z,"[a, tribute, to, songwriter, stephin, merritt,..."
1581795,1885978929,"Recording my First single ""Sooner or Later""","I am recording my first singIe, something I've...",3000.0,10.0,failed,US,$,1389395378,1389395378,...,1,2442047.0,Los Angeles,42,Pop,music/pop,False,https://www.kickstarter.com/projects/32312517/...,Kickstarter_2018-08-16T03_20_13_856Z,"[i, am, recording, my, first, singie, ,, somet..."
1581796,320197601,"Going solo! My CD ""Legendary"" will premiere on...","“Legendary” is a unique blend of 90's, R&B, po...",10000.0,81.0,failed,US,$,1391371218,1391371218,...,4,2391279.0,Denver,42,Pop,music/pop,False,https://www.kickstarter.com/projects/143905238...,Kickstarter_2018-08-16T03_20_13_856Z,"[“, legendary, ”, is, a, unique, blend, of, 90..."


In [98]:
df_nd.reset_index()

,index,id,name,blurb,goal,pledged,state,country,currency_symbol,deadline,...,backers_count,location.id,location.name,category.id,category.name,category.slug,spotlight,urls.web.project,run_id,cleanblurb
0,1581794,2061552595,Scott Fagan Sings Stephin Merritt,A tribute to songwriter Stephin Merritt (Magne...,8000.0,3565.0,failed,US,$,1391977914,...,83,2459115.0,New York,42,Pop,music/pop,False,https://www.kickstarter.com/projects/450132630...,Kickstarter_2018-08-16T03_20_13_856Z,"[a, tribute, to, songwriter, stephin, merritt,..."
1,1581795,1885978929,"Recording my First single ""Sooner or Later""","I am recording my first singIe, something I've...",3000.0,10.0,failed,US,$,1389395378,...,1,2442047.0,Los Angeles,42,Pop,music/pop,False,https://www.kickstarter.com/projects/32312517/...,Kickstarter_2018-08-16T03_20_13_856Z,"[i, am, recording, my, first, singie, ,, somet..."
2,1581796,320197601,"Going solo! My CD ""Legendary"" will premiere on...","“Legendary” is a unique blend of 90's, R&B, po...",10000.0,81.0,failed,US,$,1391371218,...,4,2391279.0,Denver,42,Pop,music/pop,False,https://www.kickstarter.com/projects/143905238...,Kickstarter_2018-08-16T03_20_13_856Z,"[“, legendary, ”, is, a, unique, blend, of, 90..."
3,1581797,944465480,Instant Burning Love,Am in need of funds to turn 9 poems into songs...,200000.0,5.0,failed,US,$,1394206075,...,1,2391279.0,Denver,42,Pop,music/pop,False,https://www.kickstarter.com/projects/194688750...,Kickstarter_2018-08-16T03_20_13_856Z,"[am, in, need, of, funds, to, turn, 9, poems, ..."
4,1581798,89956411,"PolyBlack presents ""Metanoia"" from DIN to HESE...",PolyBlack has been in studios across the Unite...,4500.0,50.0,failed,US,$,1390377600,...,1,2442047.0,Los Angeles,42,Pop,music/pop,False,https://www.kickstarter.com/projects/polyblack...,Kickstarter_2018-08-16T03_20_13_856Z,"[polyblack, has, been, in, studios, across, th..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169996,1013236,1776124488,MelMattScentsations Made with Love to Light Up...,"100% soy candles. Paraben free lotions, soaps,...",50000.0,71.0,failed,US,$,1529986596,...,4,2354141.0,Amarillo,343,Candles,crafts/candles,False,https://www.kickstarter.com/projects/melmattsm...,Kickstarter_2019-08-15T03_20_03_022Z,"[100, soy, candles, ., paraben, free, lotions,..."
169997,1013237,1364046407,Rachel McClusky's FIRST Full Length Album,Up-and-coming Chicago-based singer and songwri...,2500.0,2520.0,successful,US,$,1317313730,...,40,2379574.0,Chicago,42,Pop,music/pop,True,https://www.kickstarter.com/projects/431152386...,Kickstarter_2019-08-15T03_20_03_022Z,"[up-and-coming, chicago-based, singer, and, so..."
169998,1013238,964042215,Reckless Rockstar - clothing to illustrate epi...,Reckless rockstar is an apparel line which des...,750.0,70.0,failed,US,$,1430004523,...,2,2518097.0,Westerly,289,Textiles,art/textiles,False,https://www.kickstarter.com/projects/123938735...,Kickstarter_2019-08-15T03_20_03_022Z,"[reckless, rockstar, is, an, apparel, line, wh..."
169999,1013226,98199993,ChainDuino,Have you ever wanted to connect lots of Arduin...,2500.0,12190.0,successful,US,$,1409374800,...,146,2367105.0,Boston,52,Hardware,technology/hardware,True,https://www.kickstarter.com/projects/137045484...,Kickstarter_2019-08-15T03_20_03_022Z,"[have, you, ever, wanted, to, connect, lots, o..."


In [99]:
df_nd.loc[df_nd['name']=='Naughty Bits Enamel Pins']

,id,name,blurb,goal,pledged,state,country,currency_symbol,deadline,state_changed_at,...,backers_count,location.id,location.name,category.id,category.name,category.slug,spotlight,urls.web.project,run_id,cleanblurb
1113975,873722850,Naughty Bits Enamel Pins,by imptwitch,850.0,1047.0,successful,US,$,1533771358,1533771359,...,47,2357536.0,Austin,262,Accessories,fashion/accessories,True,https://www.kickstarter.com/projects/imptwitch...,Kickstarter_2019-08-15T03_20_03_022Z,"[by, imptwitch]"


In [100]:
# Extract the necessary columns
df2slim = df_nd[['name', 'blurb', 'pledged', 'state', 'currency_symbol', 'backers_count', 'category.name', 'urls.web.project', 'cleanblurb']]

In [6]:
# Reset the index
df2slim = df2slim.reset_index(drop=True)

In [7]:
# Convert blurb to string
df2slim['blurb'] = df2slim['blurb'].astype(str)

In [8]:
# Define a function that cleans strings: lowercase, remove select punctuation, tokenize
def clean_text(my_str):
    str_low = my_str.lower()
    remove = string.punctuation
    remove = remove.replace(".", " ").replace(",", " ").replace("'", " ")
    pattern = r"[{}]".format(remove) 
    cleanstr = re.sub(pattern, " ", str_low)
    finalword = word_tokenize(cleanstr)
    return finalword

In [9]:
# Call functions on blurb to clean the text

clean_blurb = []

for i in df2slim['blurb']:
    cleaned = clean_text(i)
    clean_blurb.append(cleaned)

In [10]:
len(clean_blurb)

1668216

In [12]:
df2slim['cleanblurb'] = clean_blurb

In [13]:
df2slim.sample(3)

,id,name,blurb,goal,pledged,state,country,currency_symbol,deadline,state_changed_at,...,backers_count,location.id,location.name,category.id,category.name,category.slug,spotlight,urls.web.project,run_id,cleanblurb
1488335,932673824,ATLAS 3D - The 3D Scanner You Print and Build ...,ATLAS 3D is a 3D printable laser scanner kit b...,3000.0,219996.0,successful,US,$,1423277296,1423277296,...,1220,2426704.0,Independence,334,DIY Electronics,technology/diy electronics,True,https://www.kickstarter.com/projects/154531538...,Kickstarter_2018-11-15T03_20_50_568Z,"[atlas, 3d, is, a, 3d, printable, laser, scann..."
1414879,905760752,ORV - Outdoor Remote Vehicle,"ORV is an all season, all electric, easy to us...",40000.0,69.0,failed,US,$,1416789580,1416789581,...,6,2487129.0,St. Paul,338,Robots,technology/robots,False,https://www.kickstarter.com/projects/137056221...,Kickstarter_2018-11-15T03_20_50_568Z,"[orv, is, an, all, season, ,, all, electric, ,..."
313098,584073517,Christmas with Rob Maletick,I am finally recording some of my favorite Chr...,8000.0,10.0,failed,US,$,1346080079,1346080079,...,1,2437920.0,Leesburg,41,Jazz,music/jazz,False,https://www.kickstarter.com/projects/162222765...,Kickstarter_2019-04-18T03_20_02_220Z,"[i, am, finally, recording, some, of, my, favo..."


In [105]:
df2slim.reset_index()

,index,name,blurb,pledged,state,currency_symbol,backers_count,category.name,urls.web.project,cleanblurb
0,1581794,Scott Fagan Sings Stephin Merritt,A tribute to songwriter Stephin Merritt (Magne...,3565.0,failed,$,83,Pop,https://www.kickstarter.com/projects/450132630...,"[a, tribute, to, songwriter, stephin, merritt,..."
1,1581795,"Recording my First single ""Sooner or Later""","I am recording my first singIe, something I've...",10.0,failed,$,1,Pop,https://www.kickstarter.com/projects/32312517/...,"[i, am, recording, my, first, singie, ,, somet..."
2,1581796,"Going solo! My CD ""Legendary"" will premiere on...","“Legendary” is a unique blend of 90's, R&B, po...",81.0,failed,$,4,Pop,https://www.kickstarter.com/projects/143905238...,"[“, legendary, ”, is, a, unique, blend, of, 90..."
3,1581797,Instant Burning Love,Am in need of funds to turn 9 poems into songs...,5.0,failed,$,1,Pop,https://www.kickstarter.com/projects/194688750...,"[am, in, need, of, funds, to, turn, 9, poems, ..."
4,1581798,"PolyBlack presents ""Metanoia"" from DIN to HESE...",PolyBlack has been in studios across the Unite...,50.0,failed,$,1,Pop,https://www.kickstarter.com/projects/polyblack...,"[polyblack, has, been, in, studios, across, th..."
...,...,...,...,...,...,...,...,...,...,...
169996,1013236,MelMattScentsations Made with Love to Light Up...,"100% soy candles. Paraben free lotions, soaps,...",71.0,failed,$,4,Candles,https://www.kickstarter.com/projects/melmattsm...,"[100, soy, candles, ., paraben, free, lotions,..."
169997,1013237,Rachel McClusky's FIRST Full Length Album,Up-and-coming Chicago-based singer and songwri...,2520.0,successful,$,40,Pop,https://www.kickstarter.com/projects/431152386...,"[up-and-coming, chicago-based, singer, and, so..."
169998,1013238,Reckless Rockstar - clothing to illustrate epi...,Reckless rockstar is an apparel line which des...,70.0,failed,$,2,Textiles,https://www.kickstarter.com/projects/123938735...,"[reckless, rockstar, is, an, apparel, line, wh..."
169999,1013226,ChainDuino,Have you ever wanted to connect lots of Arduin...,12190.0,successful,$,146,Hardware,https://www.kickstarter.com/projects/137045484...,"[have, you, ever, wanted, to, connect, lots, o..."


In [106]:
# Create dictionary to map 159 category names to 15 names
cat_dict = {'Public Art': 'Art/Visual Art/Photography',
'Illustration': 'Art/Visual Art/Photography',
'Conceptual Art': 'Art/Visual Art/Photography',
'Art': 'Art/Visual Art/Photography',
'Glass': 'Art/Visual Art/Photography',
'Sculpture': 'Art/Visual Art/Photography',
'Fine Art':'Art/Visual Art/Photography',
'Printing':'Art/Visual Art/Photography',
'Painting': 'Art/Visual Art/Photography',
'Photography': 'Art/Visual Art/Photography',
'Photo': 'Art/Visual Art/Photography',
'Fiction': 'Books/Zine/Print',
'Comic Books': 'Books/Zine/Print',
"Children's Books": 'Books/Zine/Print',
'Cookbooks': 'Books/Zine/Print',
'Graphic Novels': 'Books/Zine/Print',
'Poetry': 'Books/Zine/Print',
'Zines': 'Books/Zine/Print',
'Webcomics': 'Books/Zine/Print',
'Comics': 'Books/Zine/Print',
'Print': 'Books/Zine/Print',
'Photobooks': 'Books/Zine/Print',
'Nonfiction': 'Books/Zine/Print',
'Art Books': 'Books/Zine/Print',
'Anthologies': 'Books/Zine/Print',
'Publishing': 'Books/Zine/Print',
'Literary Journals': 'Books/Zine/Print',
'Periodicals': 'Books/Zine/Print',
'Crafts': 'Crafts/Textiles',
'Candles': 'Crafts/Textiles',
'Embroidery': 'Crafts/Textiles',
'Ceramics': 'Crafts/Textiles',
'Crochet': 'Crafts/Textiles',
'Pottery': 'Crafts/Textiles',
'Textiles': 'Crafts/Textiles',
'Weaving': 'Crafts/Textiles',
'Knitting': 'Crafts/Textiles',
'Quilts': 'Crafts/Textiles',
'Graphic Design': 'Design/Architecture',
'Civic Design': 'Design/Architecture',
'Interactive Design': 'Design/Architecture',
'Product Design': 'Design/Architecture',
'Design': 'Design/Architecture',
'Typography': 'Design/Architecture',
'Architecture': 'Design/Architecture',
'Workshops': 'Equipment/Tools/DIY',
'Fabrication Tools': 'Equipment/Tools/DIY',
'Woodworking': 'Equipment/Tools/DIY',
'DIY': 'Equipment/Tools/DIY',
'Camera Equipment': 'Equipment/Tools/DIY',
'Hardware': 'Equipment/Tools/DIY',
'DIY Electronics': 'Equipment/Tools/DIY',
'Accessories': 'Fashion/Apparel',
'Apparel': 'Fashion/Apparel',
'Footwear': 'Fashion/Apparel',
'Fashion': 'Fashion/Apparel',
'Jewelry': 'Fashion/Apparel',
'Couture': 'Fashion/Apparel',
'Childrenswear': 'Fashion/Apparel',
'Ready-to-wear': 'Fashion/Apparel',
'Restaurants': 'Food/Beverage',
'Farms': 'Food/Beverage',
'Food': 'Food/Beverage',
'Drinks': 'Food/Beverage',
'Vegan': 'Food/Beverage',
'Food Trucks': 'Food/Beverage',
"Farmer's Markets": 'Food/Beverage',
'Bacon': 'Food/Beverage',
'Small Batch': 'Food/Beverage',
'Live Games': 'Games/Gaming',
'Playing Cards': 'Games/Gaming',
'Mobile Games': 'Games/Gaming',
'Tabletop Games': 'Games/Gaming',
'Video Games': 'Games/Gaming',
'Games': 'Games/Gaming',
'Gaming Hardware': 'Games/Gaming',
'Puzzles': 'Games/Gaming',
'Audio': 'Media',
'Sound': 'Media',
'Mixed Media': 'Media',
'Radio & Podcasts': 'Media',
'Journalism': 'Media',
'Digital Art': 'Media',
'Installations': 'Media',
'Video Art': 'Media',
'Video': 'Media',
'Rock': 'Music',
'Classical Music': 'Music',
'Country & Folk': 'Music',
'R&B': 'Music',
'Pop': 'Music',
'Jazz': 'Music',
'Music': 'Music',
'Indie Rock': 'Music',
'Electronic Music': 'Music',
'Metal': 'Music',
'Hip-Hop': 'Music',
'World Music': 'Music',
'Punk': 'Music',
'Blues': 'Music',
'Nature': 'Other',
'Community Gardens': 'Other',
'Stationery': 'Other',
'Experimental': 'Other',
'Immersive': 'Other',
'Space Exploration': 'Other',
'Taxidermy': 'Other',
'Calendars': 'Other',
'Residencies': 'Other',
'Flight': 'Other',
'Latin': 'Other',
'Academic': 'Other',
'Fantasy': 'Other',
'Romance': 'Other',
'Translations': 'Other',
'Chiptune': 'Other',
'Letterpress': 'Other',
'Pet Fashion': 'Other',
'Kids': 'Other',
'Animals': 'Other',
'People': 'Other',
'Family': 'Other',
'Young Adult': 'Other',
'Faith': 'Other',
'Musical': 'Performance Art',
'Plays': 'Performance Art',
'Performance Art': 'Performance Art',
'Comedy': 'Performance Art',
'Theater': 'Performance Art',
'Performances': 'Performance Art',
'Dance': 'Performance Art',
'Festivals': 'Spaces/Places/Events',
'Literary Spaces': 'Spaces/Places/Events',
'Makerspaces': 'Spaces/Places/Events',
'Places': 'Spaces/Places/Events',
'Spaces': 'Spaces/Places/Events',
'Events': 'Spaces/Places/Events',
'Software': 'Tech/Web',
'Technology': 'Tech/Web',
'3D Printing': 'Tech/Web',
'Gadgets': 'Tech/Web',
'Robots': 'Tech/Web',
'Wearables': 'Tech/Web',
'Web': 'Tech/Web',
'Apps': 'Tech/Web',
'Animation': 'Tech/Web',
'Horror': 'TV/Film/Webseries',
'Narrative Film': 'TV/Film/Webseries',
'Shorts': 'TV/Film/Webseries',
'Film & Video': 'TV/Film/Webseries',
'Drama': 'TV/Film/Webseries',
'Television': 'TV/Film/Webseries',
'Movie Theaters': 'TV/Film/Webseries',
'Documentary': 'TV/Film/Webseries',
'Science Fiction': 'TV/Film/Webseries',
'Action': 'TV/Film/Webseries',
'Thrillers': 'TV/Film/Webseries',
'Webseries': 'TV/Film/Webseries',
'Music Videos': 'TV/Film/Webseries'}

In [107]:
# Map the new column and add to df
df2slim['newcat'] = df2slim['category.name'].map(cat_dict)

/anaconda3/envs/insight/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [108]:
df2slim.sample(3)

,name,blurb,pledged,state,currency_symbol,backers_count,category.name,urls.web.project,cleanblurb,newcat
1077146,2017 Photo Contest Magazine,Get your 2017 Shoot & Share Contest Magazine! ...,3088.0,successful,$,67,Photobooks,https://www.kickstarter.com/projects/shootands...,"[get, your, 2017, shoot, share, contest, magaz...",Books/Zine/Print
1014582,Smellin' Up the Den,Smellin’ Up the Den is a hilarious in-your-fac...,1201.0,successful,$,18,Performance Art,https://www.kickstarter.com/projects/471834546...,"[smellin, ’, up, the, den, is, a, hilarious, i...",Performance Art
1137331,A Home For Sudan,A HOME FOR SUDAN is a documentary to be filmed...,125.0,failed,$,2,Journalism,https://www.kickstarter.com/projects/ansloos/a...,"[a, home, for, sudan, is, a, documentary, to, ...",Media


In [110]:
df2slim.reset_index()

,index,name,blurb,pledged,state,currency_symbol,backers_count,category.name,urls.web.project,cleanblurb,newcat
0,1581794,Scott Fagan Sings Stephin Merritt,A tribute to songwriter Stephin Merritt (Magne...,3565.0,failed,$,83,Pop,https://www.kickstarter.com/projects/450132630...,"[a, tribute, to, songwriter, stephin, merritt,...",Music
1,1581795,"Recording my First single ""Sooner or Later""","I am recording my first singIe, something I've...",10.0,failed,$,1,Pop,https://www.kickstarter.com/projects/32312517/...,"[i, am, recording, my, first, singie, ,, somet...",Music
2,1581796,"Going solo! My CD ""Legendary"" will premiere on...","“Legendary” is a unique blend of 90's, R&B, po...",81.0,failed,$,4,Pop,https://www.kickstarter.com/projects/143905238...,"[“, legendary, ”, is, a, unique, blend, of, 90...",Music
3,1581797,Instant Burning Love,Am in need of funds to turn 9 poems into songs...,5.0,failed,$,1,Pop,https://www.kickstarter.com/projects/194688750...,"[am, in, need, of, funds, to, turn, 9, poems, ...",Music
4,1581798,"PolyBlack presents ""Metanoia"" from DIN to HESE...",PolyBlack has been in studios across the Unite...,50.0,failed,$,1,Pop,https://www.kickstarter.com/projects/polyblack...,"[polyblack, has, been, in, studios, across, th...",Music
...,...,...,...,...,...,...,...,...,...,...,...
169996,1013236,MelMattScentsations Made with Love to Light Up...,"100% soy candles. Paraben free lotions, soaps,...",71.0,failed,$,4,Candles,https://www.kickstarter.com/projects/melmattsm...,"[100, soy, candles, ., paraben, free, lotions,...",Crafts/Textiles
169997,1013237,Rachel McClusky's FIRST Full Length Album,Up-and-coming Chicago-based singer and songwri...,2520.0,successful,$,40,Pop,https://www.kickstarter.com/projects/431152386...,"[up-and-coming, chicago-based, singer, and, so...",Music
169998,1013238,Reckless Rockstar - clothing to illustrate epi...,Reckless rockstar is an apparel line which des...,70.0,failed,$,2,Textiles,https://www.kickstarter.com/projects/123938735...,"[reckless, rockstar, is, an, apparel, line, wh...",Crafts/Textiles
169999,1013226,ChainDuino,Have you ever wanted to connect lots of Arduin...,12190.0,successful,$,146,Hardware,https://www.kickstarter.com/projects/137045484...,"[have, you, ever, wanted, to, connect, lots, o...",Equipment/Tools/DIY


In [113]:
# Create separate df by category
df2music = df2slim[df2slim['newcat'] == 'Music']
df2art = df2slim[df2slim['newcat'] == 'Art/Visual Art/Photography']
df2book = df2slim[df2slim['newcat'] == 'Books/Zine/Print']
df2perform = df2slim[df2slim['newcat'] == 'Performance Art']
df2other = df2slim[df2slim['newcat'] == 'Other']
df2tech = df2slim[df2slim['newcat'] == 'Tech/Web']
df2film = df2slim[df2slim['newcat'] == 'TV/Film/Webseries']
df2games = df2slim[df2slim['newcat'] == 'Games/Gaming']
df2craft = df2slim[df2slim['newcat'] == 'Crafts/Textiles']
df2design = df2slim[df2slim['newcat'] == 'Design/Architecture']
df2diy = df2slim[df2slim['newcat'] == 'Equipment/Tools/DIY']
df2fashion = df2slim[df2slim['newcat'] == 'Fashion/Apparel']
df2food = df2slim[df2slim['newcat'] == 'Food/Beverage']
df2media = df2slim[df2slim['newcat'] == 'Media']
df2events = df2slim[df2slim['newcat'] == 'Spaces/Places/Events']

In [114]:
df2art = df2art.reset_index(drop = True)

In [115]:
df2music = df2music.reset_index(drop = True)

In [116]:
df2book = df2book.reset_index(drop = True)

In [117]:
df2perform = df2perform.reset_index(drop = True)

In [118]:
df2other = df2other.reset_index(drop = True) 

In [119]:
df2tech = df2tech.reset_index(drop = True) 

In [120]:
df2film = df2film.reset_index(drop = True) 

In [121]:
df2games = df2games.reset_index(drop = True) 

In [122]:
df2craft = df2craft.reset_index(drop = True) 

In [123]:
df2design = df2design.reset_index(drop = True) 

In [124]:
df2diy = df2diy.reset_index(drop = True) 

In [125]:
df2fashion = df2fashion.reset_index(drop = True)

In [126]:
df2food = df2food.reset_index(drop = True) 

In [127]:
df2media = df2media.reset_index(drop = True) 

In [128]:
df2events = df2events.reset_index(drop = True)

## Conduct Word2Vec and Calculate Cosine Similarity

In [36]:
# Import gensim and cosine packages
import gensim
from gensim.models import KeyedVectors
from scipy.spatial.distance import cosine

In [37]:
# Build model using Google News corpus and normalize model
norm_model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300-SLIM.bin', binary=True)
norm_model.init_sims(replace=True)

In [38]:
# For word in list, get embedding, return all vectors"""
def get_vectors_norm(listcol):
    nvectors = []
    for word in listcol: 
        try:
            nvector = norm_model[word]
            nvectors.append(nvector)
        except KeyError: 
            pass
    if len(nvectors)==0:
        return np.zeros(300)
    else:
        return sum(nvectors)/len(nvectors)

In [129]:
# apply vectors to the clean text
df2art['nvectors'] = df2art['cleanblurb'].apply(get_vectors_norm)
df2book['nvectors'] = df2book['cleanblurb'].apply(get_vectors_norm)
df2craft['nvectors'] = df2craft['cleanblurb'].apply(get_vectors_norm)
df2design['nvectors'] = df2design['cleanblurb'].apply(get_vectors_norm)
df2diy['nvectors'] = df2diy['cleanblurb'].apply(get_vectors_norm)
df2events['nvectors'] = df2events['cleanblurb'].apply(get_vectors_norm)
df2fashion['nvectors'] = df2fashion['cleanblurb'].apply(get_vectors_norm)
df2food['nvectors'] = df2food['cleanblurb'].apply(get_vectors_norm)
df2film['nvectors'] = df2film['cleanblurb'].apply(get_vectors_norm)
df2media['nvectors'] = df2media['cleanblurb'].apply(get_vectors_norm)
df2music['nvectors'] = df2music['cleanblurb'].apply(get_vectors_norm)
df2perform['nvectors'] = df2perform['cleanblurb'].apply(get_vectors_norm)
df2tech['nvectors'] = df2tech['cleanblurb'].apply(get_vectors_norm)
df2other['nvectors'] = df2other['cleanblurb'].apply(get_vectors_norm)
df2games['nvectors'] = df2games['cleanblurb'].apply(get_vectors_norm)

In [111]:
# apply and add vector to full df
df2slim['nvectors'] = df2slim['cleanblurb'].apply(get_vectors_norm)

/anaconda3/envs/insight/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [130]:
# Pickle the dfs
df2art.to_pickle('df2art_vec.pkl') 
df2book.to_pickle('df2book_vec.pkl')
df2perform.to_pickle('df2perform_vec.pkl')
df2other.to_pickle('df2other_vec.pkl')
df2tech.to_pickle('df2tech_vec.pkl') 
df2film.to_pickle('df2film_vec.pkl')
df2games.to_pickle('df2games_vec.pkl')
df2craft.to_pickle('df2craft_vec.pkl')
df2design.to_pickle('df2design_vec.pkl') 
df2diy.to_pickle('df2diy_vec.pkl')
df2fashion.to_pickle('df2fashion_vec.pkl')
df2food.to_pickle('df2food_vec.pkl')
df2media.to_pickle('df2media_vec.pkl')
df2events.to_pickle('df2events_vec.pkl')
df2music.to_pickle('df2music_vec.pkl')

In [112]:
df2slim.to_pickle('df2slim_vec.pkl')

In [133]:
df2slim.loc[df2slim['name']=='The Titan Sock | The Ultimate No-Show No-Slip Sock']

,name,blurb,pledged,state,currency_symbol,backers_count,category.name,urls.web.project,cleanblurb,newcat,nvectors
1130149,The Titan Sock | The Ultimate No-Show No-Slip ...,William Painter: The best no-show sock ever! T...,126020.0,successful,$,2129,Product Design,https://www.kickstarter.com/projects/william-p...,"[william, painter, the, best, no-show, sock, e...",Design/Architecture,"[-0.019817987, 0.011243017, -0.01235951, 0.035..."
